In [1]:
from decouple import AutoConfig

In [2]:
config = AutoConfig(search_path="..")

In [20]:
POLOGYON_API_KEY = config("POLOGYON_API_KEY")

In [22]:
# POLOGYON_API_KEY

In [5]:
import requests

In [6]:
ticker = "AAPL"
multiplier = "1"
timespan = "day"
from_date = "2023-01-09"
to_date = "2023-01-09"

In [61]:
import pytz
from datetime import datetime
from typing import Literal
from dataclasses import dataclass
from urllib.parse import urlencode

POLOGYON_API_KEY = config("POLOGYON_API_KEY")


def transform_pologyon_result(result):
    unix_timestamp = result.get('t') / 1000.0
    utc_timestamp = datetime.fromtimestamp(unix_timestamp, tz = pytz.timezone('UTC'))
    return {
        'open_price' : result['o'],
        'close_price' : result['c'],
        'high_price' : result['h'],
        'low_price' : result['l'],
        'number_of_trades' : result['n'],
        'volume' : result['v'],
        'volume_weighted_average' : result['vw'],
        'time' : utc_timestamp
    }

@dataclass
class PologyonAPIClient:
    ticker : str = "AAPL"
    multiplier : int = 1
    timespan : str = "day"
    from_date : str = "2025-07-25"
    to_date : str = "2025-07-30"
    api_key : str = ""
    adjusted : bool = True
    sort : Literal["asc","desc"] = "asc"

    def get_api_key(self):
        return self.api_key or POLOGYON_API_KEY

        
    def get_params(self):
        return {
            "adjusted" : self.adjusted,
            "sort" : self.sort
        }

    def get_headers(self):
        api_key = self.get_api_key()
        return {
            "Authorization" : f"Bearer {api_key}"
        }
    
    def generate_url(self, pass_auth= False):
        path = f"/v2/aggs/ticker/{self.ticker}/range/{self.multiplier}/{self.timespan}/{self.from_date}/{self.to_date}"
        url = f"https://api.polygon.io{path}"
        params = self.get_params()
        encoded_params = urlencode(params)
        url = f"{url}?{encoded_params}"
        if pass_auth:
            api_key = self.get_api_key()
            url += f"&apiKey={api_key}"
            
        return url

    def fetch_data(self):
        headers = self.get_headers()
        url = self.generate_url()
        response = requests.get(url, headers=headers)
        response.raise_for_status() # not 200/201
        return response.json()

    def get_stock_data(self):
        data = self.fetch_data()
        results = data['results']
        dataset = []
        for result in results:
            dataset.append(
                transform_pologyon_result(result)
            )
        return dataset
        
        

In [62]:
stock_api_client = PologyonAPIClient(ticker='GOOG', multiplier = 1)
dataset = stock_api_client.get_stock_data()
dataset

[{'open_price': 192.93,
  'close_price': 194.08,
  'high_price': 195.275,
  'low_price': 192.13,
  'number_of_trades': 382221,
  'volume': 31101847.0,
  'volume_weighted_average': 194.1131,
  'time': datetime.datetime(2025, 7, 25, 4, 0, tzinfo=<UTC>)},
 {'open_price': 194.41,
  'close_price': 193.42,
  'high_price': 194.82,
  'low_price': 191.855,
  'number_of_trades': 334933,
  'volume': 25553546.0,
  'volume_weighted_average': 193.0534,
  'time': datetime.datetime(2025, 7, 28, 4, 0, tzinfo=<UTC>)},
 {'open_price': 193.3,
  'close_price': 196.43,
  'high_price': 196.66,
  'low_price': 192.99,
  'number_of_trades': 378358,
  'volume': 31891598.0,
  'volume_weighted_average': 195.3001,
  'time': datetime.datetime(2025, 7, 29, 4, 0, tzinfo=<UTC>)},
 {'open_price': 196.3,
  'close_price': 197.44,
  'high_price': 198.4799,
  'low_price': 195.56,
  'number_of_trades': 312938,
  'volume': 21150522.0,
  'volume_weighted_average': 197.2245,
  'time': datetime.datetime(2025, 7, 30, 4, 0, tzinfo

In [63]:
# This is to test on the localhost
# stock_api_client.generate_url(pass_auth=True)

In [64]:
stock_api_client.generate_url()

'https://api.polygon.io/v2/aggs/ticker/GOOG/range/1/day/2025-07-25/2025-07-30?adjusted=True&sort=asc'